# Defining Directories and categories first print data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import pandas as pd
from keras.utils import np_utils

TRAIN_DATADIR = "/home/reinaldo/Documentos/PetImages/database/"
TEST_DATADIR = "/home/reinaldo/Documentos/PetImages/ALLDATA/"

TRAIN_DATADIR2 = "/home/reinaldo/Documentos/PetImages2/database/"
TEST_DATADIR2 = "/home/reinaldo/Documentos/PetImages2/ALLDATA/"
TRAIN_DATADIR3 = "/home/reinaldo/Documentos/PetImages3/database/"
TEST_DATADIR3 = "/home/reinaldo/Documentos/PetImages3/ALLDATA/"

CATEGORIES = ["Dog", "Cat"]
img_array = []


for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(TRAIN_DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        print(img_array)
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!

In [ ]:
print(img_array)

In [ ]:
print(img_array.shape)

In [ ]:
IMG_SIZE = 100

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
new_array2 = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.imshow(new_array2, cmap='gray')
plt.show()
print(new_array.shape)

# Create training and test data

In [ ]:
training_data = []
test_data = []
def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path_train = os.path.join(TRAIN_DATADIR,category)  # create path to dogs and cats to train
        path_test = TEST_DATADIR  # create path to dogs and cats to test
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path_train)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path_train,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num, int(img.split('.')[0])])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
    for img in tqdm(os.listdir(path_test)):  # iterate over each image per dogs and cats
        try:
            img_array = cv2.imread(os.path.join(path_test,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
            new_array2 = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
            test_data.append([new_array2, class_num,int(img.split('.')[0])])  # add this to our training_data
        except Exception as e:  # in the interest in keeping the output clean...
            pass            
           
create_training_data()

print(len(training_data))
print(len(test_data))
print(training_data)


In [ ]:
import random

random.shuffle(training_data)
random.shuffle(test_data)

# Reshape training and test data

In [ ]:
X_test = []
X_train = []
y_test = []
y_train = []
z_train = []
z_test = []

for features,label,file_name in training_data:
    X_train.append(features)
    y_train.append(label)
    z_train.append(file_name)

print(X_train[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

for features,label,file_name in test_data:
    X_test.append(features)
    y_test.append(label)
    z_test.append(file_name)

#print(X_test[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
import pickle

pickle_out = open("X_test.pickle","wb")
pickle.dump(X_test,pickle_out)

pickle_out.close()

pickle_out = open("y_test.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()

pickle_out = open("X_train.pickle","wb")
pickle.dump(X_train,pickle_out)

pickle_out.close()

pickle_out = open("y_train.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()


# Saving data on pickle files

In [ ]:
#We can always load it in to our current script, or a totally new one by doing:

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)
pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test.pickle","rb")
y_test = pickle.load(pickle_in)

# Apply data on first training machine

In [ ]:
import tensorflow as tf
#from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import keras

import pickle

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test.pickle","rb")
y_test = pickle.load(pickle_in)

y_test = np_utils.to_categorical(y_test,2)
y_train = np_utils.to_categorical(y_train,2)

X_train = X_train/255.0
X_test = X_test/255.0

model = Sequential()

model.add(Conv2D(20, (3, 3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(20, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(20))

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
model.fit(X_train, y_train, batch_size=20, epochs=5, validation_split=0.1)

print("Calculating prediction")
prediction = model.predict(X_test, batch_size=20)
print(prediction)
print("done")


In [ ]:
model2 = Sequential()

model2.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:]))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(32, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model2.add(Dense(64))

model2.add(Dense(2))
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
model2.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.1)

print("Calculating prediction")
prediction2 = model2.predict(X_test, batch_size=32)
print(prediction2)
print("done")


In [ ]:
model3 = Sequential()

model3.add(Conv2D(16, (3, 3), input_shape=X_train.shape[1:]))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(16, (3, 3)))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model3.add(Dense(10))

model3.add(Dense(2))
model3.add(Activation('softmax'))

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
model3.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.3)

print("Calculating prediction")
prediction3 = model3.predict(X_test, batch_size=16)
print(prediction3)
print("done")


# sorting the predictions without lose index 

In [ ]:
label_pred = np.insert(prediction, 2, z_test , axis =1)
label_pred2 = np.insert(prediction2, 2, z_test , axis =1)
label_pred3 = np.insert(prediction3, 2, z_test , axis =1)
print(label_pred)
print(label_pred.shape)

In [ ]:
#cat_sorte = np.sort(label_pred, axis = 0)[::-1]
cat_sort = label_pred[label_pred[:,0].argsort()]
dog_sort = label_pred[label_pred[:,1].argsort()]

cat_sort2 = label_pred2[label_pred2[:,0].argsort()]
dog_sort2 = label_pred2[label_pred2[:,1].argsort()]

cat_sort3 = label_pred3[label_pred3[:,0].argsort()]
dog_sort3 = label_pred3[label_pred3[:,1].argsort()]
print(cat_sort[::-1])
print(dog_sort[::-1])


#odenar e escolher 100 melhores de cada categoria mo

# Moving images to paths of training

In [ ]:
import shutil as st

ja_treinado ="/home/reinaldo/Documentos/PetImages/ja_treinado" 
ja_treinado2 ="/home/reinaldo/Documentos/PetImages2/ja_treinado" 
ja_treinado3 ="/home/reinaldo/Documentos/PetImages3/ja_treinado" 
for img in os.listdir(TRAIN_DATADIR+"/Cat/"):
    st.move(TRAIN_DATADIR+"/Cat/"+img,ja_treinado+"/Cat/"+img)
for img in os.listdir(TRAIN_DATADIR+"/Dog/"):
    st.move(TRAIN_DATADIR+"/Dog/"+img,ja_treinado+"/Dog/"+img)
    
for img in os.listdir(TRAIN_DATADIR2+"/Cat/"):
    st.move(TRAIN_DATADIR2+"/Cat/"+img,ja_treinado+"/Cat/"+img)
for img in os.listdir(TRAIN_DATADIR2+"/Dog/"):
    st.move(TRAIN_DATADIR2+"/Dog/"+img,ja_treinado+"/Dog/"+img)
    
for img in os.listdir(TRAIN_DATADIR3+"/Cat/"):
    st.move(TRAIN_DATADIR3+"/Cat/"+img,ja_treinado+"/Cat/"+img)
for img in os.listdir(TRAIN_DATADIR3+"/Dog/"):
    st.move(TRAIN_DATADIR3+"/Dog/"+img,ja_treinado+"/Dog/"+img)


# Moving better test images from training

In [ ]:
for i in range(100):
    #path of test to training cats
    st.move(TEST_DATADIR+str(int(cat_sort[i][2]))+".jpg",TRAIN_DATADIR+"Cat/"+str(int(cat_sort[i][2]))+".jpg")
    #path of test to training dogs
    st.move(TEST_DATADIR+str(int(dog_sort[i][2]))+".jpg",TRAIN_DATADIR+"Dog/"+str(int(dog_sort[i][2]))+".jpg")
for i in range(100):
    #path of test to training cats
    st.move(TEST_DATADIR2+str(int(cat_sort2[i][2]))+".jpg",TRAIN_DATADIR2+"Cat/"+str(int(cat_sort2[i][2]))+".jpg")
    #path of test to training dogs
    st.move(TEST_DATADIR2+str(int(dog_sort2[i][2]))+".jpg",TRAIN_DATADIR2+"Dog/"+str(int(dog_sort2[i][2]))+".jpg")
for i in range(100):
    #path of test to training cats
    st.move(TEST_DATADIR3+str(int(cat_sort3[i][2]))+".jpg",TRAIN_DATADIR3+"Cat/"+str(int(cat_sort3[i][2]))+".jpg")
    #path of test to training dogs
    st.move(TEST_DATADIR3+str(int(dog_sort3[i][2]))+".jpg",TRAIN_DATADIR3+"Dog/"+str(int(dog_sort3[i][2]))+".jpg")
    
    

# New training on the loop

In [ ]:
ite=0
while os.listdir(TEST_DATADIR) and ite<5:
    ite=ite+1
    training_data = []
    test_data = []
    create_training_data()
    print("tamanho de training data")
    print(len(training_data))
    print("tamanho de test data")
    print(len(test_data))
    random.shuffle(training_data)
    random.shuffle(test_data)

    X_test = []
    X_train = []
    y_test = []
    y_train = []
    z_train = []
    z_test = []

    for features,label,file_name in training_data:
        X_train.append(features)
        y_train.append(label)
        z_train.append(file_name)

    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

    for features,label,file_name in test_data:
        X_test.append(features)
        y_test.append(label)
        z_test.append(file_name)

    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np_utils.to_categorical(y_test,2)
    y_train = np_utils.to_categorical(y_train,2)

    X_train = X_train/255.0
    X_test = X_test/255.0

    model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.3)
    print("Calculating prediction")
    prediction = model.predict(X_test, batch_size=1)
    print("done")
    pred_index = np.argsort(prediction)

    label_pred = np.insert(prediction, 2, z_test , axis =1)

    print(label_pred)
    print(label_pred.shape)

    cat_sort = label_pred[label_pred[:,0].argsort()]
    dog_sort = label_pred[label_pred[:,1].argsort()]
    

    ja_treinado ="/home/reinaldo/Documentos/PetImages/ja_treinado" 
    for img in os.listdir(TRAIN_DATADIR+"/Cat/"):
        st.move(TRAIN_DATADIR+"/Cat/"+img,ja_treinado+"/Cat/"+img)
    for img in os.listdir(TRAIN_DATADIR+"/Dog/"):
        st.move(TRAIN_DATADIR+"/Dog/"+img,ja_treinado+"/Dog/"+img)
    if len(os.listdir(TEST_DATADIR))>100:
        for i in range(100):
            #path of test to training cats
            try:
                st.move(TEST_DATADIR+str(int(cat_sort[i][2]))+".jpg",TRAIN_DATADIR+"Cat/"+str(int(cat_sort[i][2]))+".jpg")
            #path of test to training dogs
                st.move(TEST_DATADIR+str(int(dog_sort[i][2]))+".jpg",TRAIN_DATADIR+"Dog/"+str(int(dog_sort[i][2]))+".jpg")
            except:
                pass
    else:
        
        for i in range(len(os.listdir(TEST_DATADIR))):
            #path of test to training cats
            try:
                st.move(TEST_DATADIR+str(int(cat_sort[i][2]))+".jpg",TRAIN_DATADIR+"Cat/"+str(int(cat_sort[i][2]))+".jpg")
            #path of test to training dogs
                st.move(TEST_DATADIR+str(int(dog_sort[i][2]))+".jpg",TRAIN_DATADIR+"Dog/"+str(int(dog_sort[i][2]))+".jpg")
            except:
                pass

# Loop backpropagation for Second model

In [ ]:
ite=0
while os.listdir(TEST_DATADIR2) and ite<5:
    ite=ite+1
    training_data = []
    test_data = []
    create_training_data()
    print("tamanho de training data")
    print(len(training_data))
    print("tamanho de test data")
    print(len(test_data))
    random.shuffle(training_data)
    random.shuffle(test_data)

    X_test = []
    X_train = []
    y_test = []
    y_train = []
    z_train = []
    z_test = []

    for features,label,file_name in training_data:
        X_train.append(features)
        y_train.append(label)
        z_train.append(file_name)

    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

    for features,label,file_name in test_data:
        X_test.append(features)
        y_test.append(label)
        z_test.append(file_name)

    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np_utils.to_categorical(y_test,2)
    y_train = np_utils.to_categorical(y_train,2)

    X_train = X_train/255.0
    X_test = X_test/255.0

    model2.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.3)
    print("Calculating prediction")
    prediction2 = model2.predict(X_test, batch_size=1)
    print("done")
    pred_index = np.argsort(prediction2)

    label_pred2 = np.insert(prediction2, 2, z_test , axis =1)

    print(label_pred2)
    print(label_pred2.shape)

    cat_sort2 = label_pred2[label_pred2[:,0].argsort()]
    dog_sort2 = label_pred2[label_pred2[:,1].argsort()]
    

    ja_treinado ="/home/reinaldo/Documentos/PetImages/ja_treinado" 
    for img in os.listdir(TRAIN_DATADIR2+"/Cat/"):
        st.move(TRAIN_DATADIR2+"/Cat/"+img,ja_treinado+"/Cat/"+img)
    for img in os.listdir(TRAIN_DATADIR2+"/Dog/"):
        st.move(TRAIN_DATADIR2+"/Dog/"+img,ja_treinado+"/Dog/"+img)
    if len(os.listdir(TEST_DATADIR2))>100:
        for i in range(100):
            #path of test to training cats
            try:
                st.move(TEST_DATADIR2+str(int(cat_sort2[i][2]))+".jpg",TRAIN_DATADIR2+"Cat/"+str(int(cat_sort2[i][2]))+".jpg")
            #path of test to training dogs
                st.move(TEST_DATADIR2+str(int(dog_sort2[i][2]))+".jpg",TRAIN_DATADIR2+"Dog/"+str(int(dog_sort2[i][2]))+".jpg")
            except:
                pass
    else:
        
        for i in range(len(os.listdir(TEST_DATADIR2))):
            #path of test to training cats
            try:
                st.move(TEST_DATADIR2+str(int(cat_sort2[i][2]))+".jpg",TRAIN_DATADIR2+"Cat/"+str(int(cat_sort2[i][2]))+".jpg")
            #path of test to training dogs
                st.move(TEST_DATADIR2+str(int(dog_sort2[i][2]))+".jpg",TRAIN_DATADIR2+"Dog/"+str(int(dog_sort2[i][2]))+".jpg")
            except:
                pass

# Loop backpropagation for Model 3

In [ ]:
ite=0
while os.listdir(TEST_DATADIR3) and ite<5:
    ite=ite+1
    training_data = []
    test_data = []
    create_training_data()
    print("tamanho de training data")
    print(len(training_data))
    print("tamanho de test data")
    print(len(test_data))
    random.shuffle(training_data)
    random.shuffle(test_data)

    X_test = []
    X_train = []
    y_test = []
    y_train = []
    z_train = []
    z_test = []

    for features,label,file_name in training_data:
        X_train.append(features)
        y_train.append(label)
        z_train.append(file_name)

    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

    for features,label,file_name in test_data:
        X_test.append(features)
        y_test.append(label)
        z_test.append(file_name)

    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np_utils.to_categorical(y_test,2)
    y_train = np_utils.to_categorical(y_train,2)

    X_train = X_train/255.0
    X_test = X_test/255.0

    model3.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.3)
    print("Calculating prediction")
    prediction = model3.predict(X_test, batch_size=1)
    print("done")
    pred_index = np.argsort(prediction3)

    label_pred = np.insert(prediction3, 2, z_test , axis =1)

    print(label_pred)
    print(label_pred.shape)

    cat_sort = label_pred[label_pred[:,0].argsort()]
    dog_sort = label_pred[label_pred[:,1].argsort()]
    

    ja_treinado ="/home/reinaldo/Documentos/PetImages/ja_treinado" 
    for img in os.listdir(TRAIN_DATADIR3+"/Cat/"):
        st.move(TRAIN_DATADIR+"/Cat/"+img,ja_treinado+"/Cat/"+img)
    for img in os.listdir(TRAIN_DATADIR3+"/Dog/"):
        st.move(TRAIN_DATADIR3+"/Dog/"+img,ja_treinado+"/Dog/"+img)
    if len(os.listdir(TEST_DATADIR3))>100:
        for i in range(100):
            #path of test to training cats
            try:
                st.move(TEST_DATADIR3+str(int(cat_sort[i][2]))+".jpg",TRAIN_DATADIR3+"Cat/"+str(int(cat_sort[i][2]))+".jpg")
            #path of test to training dogs
                st.move(TEST_DATADIR3+str(int(dog_sort[i][2]))+".jpg",TRAIN_DATADIR3+"Dog/"+str(int(dog_sort[i][2]))+".jpg")
            except:
                pass
    else:
        
        for i in range(len(os.listdir(TEST_DATADIR3))):
            #path of test to training cats
            try:
                st.move(TEST_DATADIR3+str(int(cat_sort[i][2]))+".jpg",TRAIN_DATADIR3+"Cat/"+str(int(cat_sort[i][2]))+".jpg")
            #path of test to training dogs
                st.move(TEST_DATADIR3+str(int(dog_sort[i][2]))+".jpg",TRAIN_DATADIR3+"Dog/"+str(int(dog_sort[i][2]))+".jpg")
            except:
                pass

In [ ]:
print("Your network is working properly")